In [1]:
import pandas as pd
import glob

In [2]:
cleaned_fnames = sorted(glob.glob("../*cleaned.csv"))
cleaned_fnames = cleaned_fnames[1:]

In [3]:
cleaned_fnames

['..\\london_cleaned.csv',
 '..\\seoul_cleaned.csv',
 '..\\washington_cleaned.csv']

In [4]:
data = [pd.read_csv(fname, index_col=False) for fname in cleaned_fnames]
data = [df[df.columns[1:]] for df in data]
locations = ['london', 'seoul', 'washington_dc']
for i, df in enumerate(data):
    if i == 1:
        df['date'] = pd.to_datetime(df.date, dayfirst=True)
    else:
        df['date'] = pd.to_datetime(df.date)
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month  
    df['day'] = df.date.dt.day
    if 'hr' not in df.columns:
        df['hr'] = df.date.dt.hour
    df['location'] = locations[i]
    df['date'] = pd.to_datetime(df.date).dt.date

data[1].season = ['fall' if s == 'autumn' else s for s in data[1].season]

In [5]:
for i in range(3):
    seasons = 'spring,summer,fall,winter'.split(',')
    for s in seasons:
        print(locations[i], s,  data[i][data[i].season == s].month.unique())

london spring [3 4 5]
london summer [6 7 8]
london fall [ 9 10 11]
london winter [ 1  2 12]
seoul spring [3 4 5]
seoul summer [6 7 8]
seoul fall [ 9 10 11]
seoul winter [12  1  2]
washington_dc spring [ 1  2  3 12]
washington_dc summer [3 4 5 6]
washington_dc fall [6 7 8 9]
washington_dc winter [ 9 10 11 12]


In [6]:
washington_seasons = []
for i in data[2].month:
    if i in [3,4,5]:
        washington_seasons.append('spring')
    elif i in [6,7,8]:
        washington_seasons.append('summer')
    elif i in [9,10,11]:
        washington_seasons.append('fall')
    else:
        washington_seasons.append('winter')
        
data[2].season = washington_seasons

In [7]:
joined_df = pd.concat(data)
joined_df.date = pd.to_datetime(joined_df.date)
joined_df['day_of_week'] = joined_df.date.dt.dayofweek
joined_df['is_weekend'] = 1*(joined_df.day_of_week >= 5)
joined_df.head()

joined_df.to_csv('../combined_dataset_cleaned.csv', index=False)